In [31]:
# 多个输入通道
# 彩色图片可能有RGB三个通道
# 转化为灰度会丢失信息
# 每个通道都有一个自己的卷积核, 最后是将所有通道卷积结果按元素相加得到最后的结果

# 多个输入通道
#   - 输入X: (c_i) x (n_h) x (n_w)
#   - Kernel W: (c_i) x (k_h) x (k_w)
#   - 输出Y: (m_h) x (m_w)
# 这里多了一个维度c_i, 表示为第i个channel通道
# 输出一个单通道, 因为最后将每一个通道的结果按元素相加
# 表示为: sigma(i=0 to c_i) (X_i,:,:) * (W_i,:,:)
# 意思就是将所有channel做卷积, 最后求和

# 多个输出通道
# 无论有多少输入通道, 到目前为止我们只用到单输出通道
# 我们可以有多个三维卷积核, 每个核生成一个输出通道
#   - 输入X: (c_i) x (n_h) x (n_w)
#   - Kernel W: (c_o) x (c_i) x (k_h) x (k_w)
#   - 输出Y: (c_o) x (m_h) x (m_w)
# 意思是说: 对每一个输入X, 把它对应的那个输出铜通道的kernel拿出来, 就会得到一个对应的输出通道
# 然后把每一个输出通一一做运算, 这里的c_o是输出通道数, 意思就是把他concate起来(也就是将每个输出一层叠一层), 就能得到输出了
# 表示为 (Y_i,:,:) = X * (W_i,:,:) for i = 1,...,c_0


# 区别
# 多输入通道是不是每个通道有自己的卷积核, 最后将得到的结果matrix加起来
# 多输出通道是不是就是还是一样, 但是最后不加起来, 而是将这些得到的结果matrix叠起来, 成为一个新的维数

# 多个输入和输出通道
# 每个输出通道可以识别特定的模式 (因为是叠起来的, 所以不融合) 
# 举例: 一张猫的图片可以有颜色, 角度, 纹理等模式被识别出来; 比如这里的输出通道为6
# 输入通道核识别并组合输入中的模式 (因为是加起来的, 通过权重融合, 所以得到一个合体)
# 举例: 把这6个输出通道丢给下一个层, 下一个层把每个模式识别并按权重组合, 就得到一个组合的模式识别
# 也就是说底层会识别一些纹理什么, 越往上就会将纹理组合起来, 成为形状

In [32]:
# 1x1卷积层
# k_h = k_w = 1是一个受欢迎的选择; 他不识别空间模式, 只是融合通道
# 也就是将输入拉成一个向量, 假如输入是(n_h)x(n_w)的向量, 拉成一个长为(n_h n_w)的向量, 这样就丧失了空间信息
# 将channel数拉成一个feature数, Kernel是一个(c_o)x(c_i)
# 所以就是一个输入形状为(n_h n_w)x(c_i), 权重为 (c_o)x(c_i)的全连接层

# 举例
# 你有一个权重集，对于每个输出通道，都有一个对应的权重集。假设你只有一个输出通道，那么你将有一组权重 w_a, w_b, w_c
# 对于输出矩阵中的每一个位置(i,j), 计算方式是 w_a * A[i,j] + w_b * B[i,j] + w_c * C[i,j]
# 最后输出的也是一个3x3的矩阵, 其中每个元素是通过对应位置的三个通道的像素值进行加权求和得到的

# 二维卷积层
# 输入X: (c_i) x (n_h) x (n_w)
# 核W: (c_o) x (c_i) x (k_h) x (k_w)
# 偏差B: (c_o) x (c_i)
# 输出Y: (c_o) x (m_h) x (m_w)

In [33]:
# 总结
# 输出通道数是卷积层的超参数
# 每一个输入通道都有独立的二维卷积层, 所有通道结果相加得到一个输出通道结果的矩阵
# 每个输出通道都有独立的三维卷积层

In [34]:
# 多输入多输出通道
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K): # 多输入的时候怎么算互相关
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))
# 因为这里X,K是个多输入的, 用zip绑定然后一起遍历每一个channel的x,k, 放入卷积, 最后按元素求和(输入是融合) 

In [35]:
# 验证互相关运算
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [36]:
# 计算多个通道的输出的互相关函数
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0) # 输出是叠加
# 这里是对每一个输出通道的K, 拿出一个k, 这里的k是一个3D的tensor, 这个k和完整的X做互相关运算
# 用stack新建维度0, 然后计算完的结果一个一个堆起来

# K是一个三维的一个2x2x2
# 新建一个维度0, 将K叠到一起, 就称为输出通道为3, 输入通道为2, 高和宽是2
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [37]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [42]:
# 1x1卷积
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0] # 输出的通道数
    X = X.reshape((c_i, h * w)) # 将X变为(ci x (nh * nw))的矩阵, 这里拉平了所有的空间性
    K = K.reshape((c_o, c_i)) # 将K变为(co x ci)的矩阵
    Y = torch.matmul(K, X) #最后相乘
    return Y.reshape((c_o, h, w)) #最后reshape回去

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6 # 几乎完全一样